In [ ]:
import pandas as pd
import logging
from sqlalchemy import create_engine
import jieba

In [ ]:
restaurant_df=pd.read_csv("C:/Users/kiddc/jupyter_資料/推薦餐廳_有加地區_且有數值.csv")

In [ ]:
restaurant_df=restaurant_df.drop('Unnamed: 0',axis=1)

In [ ]:
restaurant_df['res_name']=restaurant_df['res_name'].str.replace(' ','')

In [ ]:
restaurant_df

In [ ]:
restaurant_df['price']=pd.qcut(restaurant_df['price'], 7,labels=['超實惠','平民','實惠','普通','貴','高價','高端'])
restaurant_df

In [ ]:
restaurant_df['score']=pd.qcut(restaurant_df['score'], 8,labels=['難吃','差','普通','好吃','美味','一星級','二星級','三星級'])
restaurant_df

In [ ]:
# 建立function作jieba分詞
def lcut(article):
    segment=[]
    segs = jieba.lcut(article) # jiaba.lcut()  
    for seg in segs:
        if len(seg)>1 and seg!='\r\n':
            segment.append(seg)
    return segment

In [ ]:
# 基於TextRank算法的關鍵詞抽取(僅名詞、地名、)
import jieba.analyse as analyse
count = 0
restaurant_df['feature'] = restaurant_df.content.apply(lcut)\
                .apply(lambda x : " ".join(x))\
                .apply(lambda x: analyse.textrank(x, topK=10, withWeight=False, allowPOS=('n','a','an')))
restaurant_df

In [ ]:
df1=restaurant_df.drop(['Unnamed: 0','res_id','address','content','clean_add_num'],axis=1)#刪除不需要的欄位

In [ ]:
df1.to_csv("11_28_33449_10特徵在一欄(去些欄跑word2vec).txt",sep='\t',index=False) #存檔成txt

In [ ]:
df2=pd.read_csv('C:/Users/kiddc/jupyter_程式/11_28_33449_10特徵在一欄.csv') #測試是否存好檔

In [ ]:
df2

In [ ]:
from gensim.models import word2vec

In [ ]:
jieba.set_dictionary('C:/Users/kiddc/OneDrive/桌面/tibame/Textming/jiebaData/dict.txt.big')

In [ ]:
sentences= word2vec.LineSentence('11_28_33449_10特徵在一欄(去些欄跑word2vec).txt')  #使用txt檔去訓練模型

In [ ]:
import time
start_time = time.time()
model =word2vec.Word2Vec(sentences, sg=3,vector_size=250, window=10,workers=3, min_count=1)
print("-----Spend %s Second-----" %(time.time() - start_time))

In [ ]:
model

In [ ]:
model.save('11_28_33449_10特徵在一欄(去些欄跑word2vec).model')

In [ ]:
model.corpus_total_words

In [ ]:
model.wv.most_similar('舒果(板橋雙十店)')

In [ ]:
model.wv.similarity('款待手路菜','北平半畝園')

In [ ]:
model.wv.most_similar('小貴族灸燒牛排')